In [47]:
# Imports

import math
import numpy as np
from matplotlib import pyplot as plt
from numpy import random as rnd
import warnings,datetime,os,calendar,csv,time

import tensorflow as tf
import pandas as pd
import seaborn as sns

from keras.models import Model,Sequential
from keras.layers import Dense,LSTM,Conv2D,Dropout,BatchNormalization,Input,Concatenate,Add,Activation,MaxPooling2D,AveragePooling2D
import keras.backend as K

from sklearn import preprocessing as pp

from sklearn.cluster import KMeans,MeanShift
from sklearn.dummy import DummyClassifier,DummyRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier,AdaBoostRegressor,RandomForestClassifier,RandomForestRegressor
from sklearn.linear_model import BayesianRidge,Lasso,LinearRegression,SGDClassifier,SGDRegressor
from sklearn.mixture import BayesianGaussianMixture,GaussianMixture
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor,RadiusNeighborsClassifier,RadiusNeighborsRegressor,NearestNeighbors
from sklearn.manifold import Isomap,TSNE
from sklearn.model_selection import KFold,StratifiedKFold,train_test_split
from sklearn.svm import LinearSVC,LinearSVR
from sklearn.neural_network import BernoulliRBM,MLPClassifier,MLPRegressor
from sklearn.decomposition import FactorAnalysis,KernelPCA,PCA,MiniBatchSparsePCA,FastICA
from sklearn.preprocessing import CategoricalEncoder,KBinsDiscretizer,LabelEncoder,MinMaxScaler,OneHotEncoder,StandardScaler

from xgboost import XGBClassifier,XGBRegressor

import gym
import pygame
from pygame.locals import *

import pickle,h5py,json

import pandas_datareader as pdr
import urllib
from bs4 import BeautifulSoup
import iexfinance as iex
from iexfinance.stocks import get_historical_data
from scipy.signal import resample,correlate
from scipy import fftpack
from lmfit import Model as lmmodel

warnings.filterwarnings('ignore')
sns.set()
plt.xkcd()

In [2]:
def nan_helper(y):
    return np.isnan(y), lambda z: z.nonzero()[0]

def make_dateaxis(start_date,end_date):
    datelist = []
    d = start_date
    delta = datetime.timedelta(days=1)
    while d <= end_date:
        if (d.weekday()==5 or d.weekday()==6):
            d += delta
            continue
        datelist.append(d)
        d += delta
    datelist = np.stack(datelist,axis=0)
    return datelist

def unpack_index(DF,datelist,idx,fname,verbose=False):
    index_flist = []
    first_datapoint_found = False
    for d in list(datelist):
        try:
            index_f = DF[idx][fname].loc[d.strftime('%Y-%m-%d')]
        except:
            index_f = 0
        if index_f==0 and first_datapoint_found:
            index_f = np.nan
        if index_f!=0 and not first_datapoint_found:
            first_datapoint_found = True
        if verbose: print('Date %s: Open Value %.3f'%(d.strftime('%Y-%m-%d'),index_f))
        index_flist.append(index_f)
    index_flist = np.stack(index_flist,axis=0)
    nans,x = nan_helper(index_flist)
    index_flist[nans]= np.interp(x(nans),x(~nans),index_flist[~nans])
    return index_flist

In [3]:
start_date = datetime.datetime(2015,1,1)
end_date = datetime.datetime(2017,1,1)

with open('Documents\stock_symb.txt','r') as f:
    lines = f.readlines()
content = [line.strip() for line in lines]
tickers = [content[i] for i in list(np.random.permutation(np.arange(len(content))))]

In [7]:
ticker_set = tickers[:50]

DATA = []

ctr = 0
for ticker in ticker_set:

    if ctr%10==0: print('Processing %d/%d'%(ctr,len(ticker_set)))
    ctr += 1
    try:
        df = get_historical_data(ticker,start=start_date,end=end_date,output_format='pandas')
    except:
        continue
    
    data = df['open'].values
    DATA.append(data)

Processing 0/50
Processing 10/50
Processing 20/50
Processing 30/50
Processing 40/50


In [9]:
ctr = 0
for data in DATA:
    if len(data)!=504 or np.isnan(np.array(data)).sum()!=0:
        del DATA[ctr]
    ctr += 1
for data in DATA:
    print('Length = %d, #nans = %d'%(len(data),np.isnan(np.array(data)).sum()))

Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 491, #nans = 33
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0
Length = 504, #nans = 0


In [17]:
DDATA = []
for data in DATA:
    DDATA.append(np.diff(data))

In [19]:
lookback = 100

IN = []
NEXT = []

for data in DDATA:
    
    for k in range(lookback,data.shape[0]):
        
        IN.append(data[k-lookback:k])
        NEXT.append(data[k])
        
D = np.stack(IN,axis=0)
L = np.stack(NEXT,axis=0)

In [34]:
Xtrain,Xtest,ytrain,ytest = train_test_split(D,L,test_size=0.3,shuffle=True)

In [67]:
in0 = Input(shape=(lookback,1,))
lstm0 = LSTM(units=15,activation='relu',return_sequences=False,return_state=False)
dense0 = Dense(units=1,activation='sigmoid')

x = lstm0(in0)
out0 = dense0(x)

model = Model(in0,out0)
model.compile(optimizer='adam',loss='mse',metrics=['accuracy'])

In [68]:
model.fit(Xtrain.reshape(-1,lookback,1),ytrain,batch_size=5,epochs=5,validation_split=0.2)

Train on 7214 samples, validate on 1804 samples
Epoch 1/5
7214/7214 [==============================] - 106s 15ms/step - loss: nan - acc: 1.3862e-04 - val_loss: nan - val_acc: 0.0000e+00
Epoch 2/5
7214/7214 [==============================] - 126s 17ms/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 3/5
7214/7214 [==============================] - 92s 13ms/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 4/5
7214/7214 [==============================] - 102s 14ms/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 5/5
7214/7214 [==============================] - 94s 13ms/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
